# Iterated Prisoner's Dilemma On A Network
## Imports and Configuration

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
from Prison import (
    ImitationStrategy,
    NetworkSimulation,
    TitForTatStrategy,
    experiment,
    FermiStrategy,
)

In [ ]:
%matplotlib inline
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams["animation.embed_limit"] = 500

## Helper function

In [ ]:
def calculate_percolation_metrics(simulation):
    """
    Calculates the Order Parameter (S_max) and Susceptibility (Chi).
    S_max: Fraction of nodes in the largest cluster of cooperators.
    Chi: Average size of non-giant cooperative clusters (measure of fluctuations).
    """
    state = simulation._get_state()
    # Identify Cooperator nodes (Action 0 is typically Cooperation in these libs)
    coop_nodes = [n for n, action in state.items() if action == 0]
    N = simulation.graph.number_of_nodes()

    # If no cooperators exist, order and susceptibility are 0
    if not coop_nodes:
        return 0.0, 0.0, 0.0, 0.0

    # Create the subgraph of ONLY cooperators
    G_coop = simulation.graph.subgraph(coop_nodes)

    # Get all connected components (clusters)
    components = list(nx.connected_components(G_coop))

    if not components:
        return 0.0, 0.0, 0.0, 0.0

    # Sort by size (largest to smallest)
    sizes = sorted([len(c) for c in components], reverse=True)

    N_clusters = len(sizes)

    S_average = np.mean(sizes)

    # 1. Order Parameter: Size of largest cluster normalized by N
    S_max = sizes[0]

    # 2. Susceptibility: Second moment of the distribution of small clusters
    other_sizes = sizes[1:]  # Exclude the giant component

    if not other_sizes:
        chi = 0.0
    else:
        # Formula: Sum(s^2 * n_s) / Sum(s * n_s)
        # Since 'other_sizes' is a list of ALL sizes, summing s^2 iterates over every node in those clusters
        numerator = sum(s**2 for s in other_sizes)
        denominator = sum(other_sizes)
        chi = numerator / denominator if denominator > 0 else 0.0

    return S_max, chi, S_average, N_clusters

## Experiments

In [ ]:
# --- CONFIGURATION FOR RQ3 ---
N_SIDE = 20  # 900 Agents (Good balance of speed vs accuracy)
STEPS = 100  # Allow system to settle into a steady state
TRIALS = 5  # Higher trials to smooth the susceptibility curve
# Focus the sweep on the critical region (b/c between 0 and 2 is usually critical for simple grids)
BETAS = np.linspace(0.0, 10.0, 21)
BETAS = np.linspace(0.0, 3.0, 31)

results = []

print(f"Starting Simulation for: {N_SIDE}x{N_SIDE} Grid, {TRIALS} Trials per Beta...")

for beta in tqdm(BETAS):
    # Setup Payoff Parameters
    # b/c ratio is the main control parameter
    kbar = 4  # Average degree for grid
    c = 1.0
    b = beta * kbar * c  # Scaling b relative to neighbors

    # Standard Prisoner's Dilemma Payoffs
    pm = {
        ("C", "C"): (b - c, b - c),
        ("C", "D"): (-c, b),
        ("D", "C"): (b, -c),
        ("D", "D"): (0, 0),
    }

    pm = {
        ("C", "C"): (1.0, 1.0),
        ("C", "D"): (0.0, beta),
        ("D", "C"): (beta, 0.0),
        ("D", "D"): (0, 0),
    }


    s_max_list = []
    s_mean_list = []
    chi_list = []
    N_clusters_list = []

    for _ in range(TRIALS):
        sim = NetworkSimulation(
            kind="grid",
            rounds=STEPS,
            n=N_SIDE * N_SIDE,
            payoff_matrix=pm,
            strategy=ImitationStrategy,
            strategy_kwargs={},
            # FIX: Window=1 for immediate imitation (High "Temperature"/Reactivity)
            history_window=10,
            store_history=True,  # Optimization: Don't store full history if not needed
            store_snapshots=False,
        )

        # Run simulation
        sim.run()
        # sim.run_until_attractor(max_steps=STEPS, check_every=100, store_cycle_states=False)

        # Measure Order and Susceptibility
        s_max, chi, s_mean, N_clusters = calculate_percolation_metrics(sim)
        s_mean_list.append(s_mean)
        s_max_list.append(s_max)
        chi_list.append(chi)
        N_clusters_list.append(N_clusters)

    results.append(
        {
            "beta": beta,
            "b_over_c": b / c,
            "S_max_mean": np.mean(s_max_list),
            "S_max_std": np.std(s_max_list),
            "chi_mean": np.mean(chi_list),
            "chi_std": np.std(chi_list),
            "s_mean": np.mean(s_mean_list),
            "s_mean_std": np.std(s_mean_list),
            "N_clusters": np.mean(N_clusters_list),
            "N_clusters_std": np.std(N_clusters_list),
        }
    )

df_res = pd.DataFrame(results)

In [ ]:
display(df_res.head())

### Visualization

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 7))

colors = {
    "S_max": "tab:blue",
    "S_mean": "tab:red",
    "N_clusters": "tab:green",
}

ax1.set_xlabel(r"Temptation parameter ($\beta$)", fontsize=14)
ax1.set_ylabel(r"Cluster Size ($S$)", fontsize=14)
ax1.grid(True, which="both", alpha=0.3)

ax1.plot(
    df_res["beta"],
    df_res["S_max_mean"],
    color=colors["S_max"],
    marker="o",
    lw=2,
    label=r"$S_\text{max}$",
)
ax1.fill_between(
    df_res["beta"],
    df_res["S_max_mean"] - df_res["S_max_std"],
    df_res["S_max_mean"] + df_res["S_max_std"],
    color=colors["S_max"],
    alpha=0.2,
)


ax1.plot(
    df_res["beta"],
    df_res["s_mean"],
    color=colors["S_mean"],
    marker="o",
    lw=2,
    label=r"$\bar S$",
)
ax1.fill_between(
    df_res["beta"],
    df_res["s_mean"] - df_res["s_mean_std"],
    df_res["s_mean"] + df_res["s_mean_std"],
    color=colors["S_mean"],
    alpha=0.1,
)

ax2 = ax1.twinx()
ax2.set_ylabel(r"Number of clusters ($N$)", fontsize=14)

ax2.plot(
    df_res["beta"],
    df_res["N_clusters"],
    color=colors["N_clusters"],
    marker="o",
    linestyle="--",
    lw=2,
    label=r"$N_\text{clusters}$",
)
ax2.fill_between(
    df_res["beta"],
    df_res["N_clusters"] - df_res["N_clusters_std"],
    df_res["N_clusters"] + df_res["N_clusters_std"],
    color=colors["N_clusters"],
    alpha=0.1,
)


plt.title(
    f"Phase Transition of Cooperative Clusters\nGrid {N_SIDE}x{N_SIDE}, Imitation Strategy",
    fontsize=16,
)
fig.legend(
    loc="lower center",
    bbox_to_anchor=(0.5, -0.06),
    ncol=3,
    fontsize=12,
)
fig.tight_layout()

plt.show()

In [ ]:
print(BETAS)

## Individual experiments

In [ ]:
save = False

for beta in tqdm([0.5, 0.8, 1.0, 1.1, 2.0, 2.6]):
# for beta in tqdm([1.0, 1.02, 1.05, 1.1, 1.2]):
    anim_path = f"Tim_Imitation_beta_{beta}.gif"
    kbar = 4
    c = 1.0
    b = beta * kbar * c

    pm = {
        ("C", "C"): (1.0, 1.0),
        ("C", "D"): (0.0, beta),
        ("D", "C"): (beta, 0.0),
        ("D", "D"): (0, 0),
    }

    gif = experiment(
        NetworkSimulation,
        strategy_class=ImitationStrategy,
        strategy_kwargs={},
        steps=STEPS,
        seed=int(42 + beta * 5),  # Fixed seed for reproducibility
        interval=75,  # Speed of animation (ms per frame)
        payoff_matrix=pm,
        title=f"Critical Phase Transition (beta = {beta:.2f})",
        kind="grid",
        n=N_SIDE * N_SIDE,  # 2500 agents
        is_grid=True,  # Render as a pixel grid
        save_gif=save,
        gif_path=anim_path,
    )

    if save:
        print(f"Animation saved to {anim_path}")
    display(gif)